# Perceptrón

In [1]:
import numpy as np
from utils import predicted_report
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import to_categorical
import pickle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from utils import predicted_report, SEED

Using TensorFlow backend.


In [2]:
with open('./rebalanced_data/train_set_oss.pickle', mode='rb') as f:
    train_oss = pickle.load(f)

In [3]:
with open('./rebalanced_data/test_set.pickle', mode='rb') as f:
    test_set = pickle.load(f)

In [4]:
with open('./rebalanced_data/k_folds_oss.pickle', mode='rb') as f:
    k_folds = pickle.load(f)

In [5]:
y_train = to_categorical([1 if y == 1 else 0 for y in train_oss['y']], 2)

In [6]:
def create_model(init, optimizer):
    model = Sequential()
    model.add(Dense(30, input_dim=29, activation='relu', kernel_initializer=init
                   ))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='sigmoid'))   
    
    model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    return model

In [7]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [8]:
np.random.seed(SEED)

tuned_parameters = {'optimizer': np.array(['rmsprop']), 
                    'init': np.array(['zeros', 'ones',
                                      'TruncatedNormal', 'glorot_normal', 'he_normal', 'lecun_uniform'
                                     ]), 
                    'epochs': np.array([100, 1000]), 
                    'batch_size': np.array([1000, 4000, 8000])}

grid = GridSearchCV(estimator=model, 
                    param_grid=tuned_parameters, 
                    cv=5,
                    n_jobs=-1)

grid_result = grid.fit(np.array(train_oss['x']), y_train)

In [9]:
print(grid_result.best_params_)
print()
print(grid_result.cv_results_['mean_test_score'])
print()
print(grid_result.cv_results_['std_test_score'])

{'batch_size': 1000, 'optimizer': 'rmsprop', 'epochs': 1000, 'init': 'TruncatedNormal'}

[ 0.04875836  0.85735344  0.90826623  0.91110103  0.91302869  0.90917338
  0.04875836  0.90463771  0.94494841  0.94443814  0.9436444   0.9437578
  0.15126432  0.7868239   0.89329855  0.89086065  0.89086065  0.88536114
  0.15126432  0.90769929  0.93389275  0.94069622  0.93723778  0.93752127
  0.10001134  0.7291076   0.86086859  0.83558228  0.81159995  0.81273388
  0.04875836  0.89295839  0.92924369  0.93264542  0.93264543  0.93071777]

[ 0.09750982  0.01440586  0.03643103  0.02241034  0.0228759   0.03320086
  0.09750982  0.03132578  0.01732896  0.01666596  0.01796936  0.01484245
  0.3025072   0.09707717  0.02516841  0.02879989  0.03328637  0.03906611
  0.3025072   0.01619211  0.02074463  0.01755886  0.02129257  0.02070185
  0.2000085   0.15869513  0.03798083  0.04821079  0.06826131  0.08177348
  0.09750982  0.01891998  0.02650377  0.02284807  0.02395535  0.02233517]


Se entrena un modelo con todo el conjunto de entrenamiento con los parámetros del Grid-Search.

In [10]:
model_final = create_model(init=grid_result.best_params_['init'], optimizer=grid_result.best_params_['optimizer'])
model_final.fit(np.array(train_oss['x']), y_train,
                  epochs=grid_result.best_params_['epochs'],
                  batch_size=grid_result.best_params_['batch_size'])

Epoch 1/1000
8819/8819 [==============================] - 0s - loss: 0.6718 - acc: 0.6089     
Epoch 2/1000
8819/8819 [==============================] - 0s - loss: 0.6086 - acc: 0.7766     
Epoch 3/1000
8819/8819 [==============================] - 0s - loss: 0.5579 - acc: 0.8282     
Epoch 4/1000
8819/8819 [==============================] - 0s - loss: 0.5123 - acc: 0.8450     
Epoch 5/1000
8819/8819 [==============================] - 0s - loss: 0.4731 - acc: 0.8636     
Epoch 6/1000
8819/8819 [==============================] - 0s - loss: 0.4378 - acc: 0.8712     
Epoch 7/1000
8819/8819 [==============================] - 0s - loss: 0.4084 - acc: 0.8745     
Epoch 8/1000
8819/8819 [==============================] - 0s - loss: 0.3778 - acc: 0.8840     
Epoch 9/1000
8819/8819 [==============================] - 0s - loss: 0.3537 - acc: 0.8881     
Epoch 10/1000
8819/8819 [==============================] - 0s - loss: 0.3320 - acc: 0.8923     
Epoch 11/1000
8819/8819 [========================

In [11]:
y_test = [1 if y == 1 else 0 for y in test_set['y']]

score = model_final.evaluate(np.array(test_set['x']), to_categorical(y_test, 2), batch_size=128)

 128/2211 [>.............................] - ETA: 0s

In [12]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0987608782485
Test accuracy: 0.966078697449


In [13]:
results_to_vals = np.vectorize(lambda x: 'legit' if x == 1 else 'phishing')
predicted = model_final.predict_classes(np.array(test_set['x']))
predicted_report(results_to_vals(y_test), results_to_vals(predicted))

  32/2211 [..............................] - ETA: 1sPredicted  legit  phishing   All
Actual                          
legit       1246        25  1271
phishing      50       890   940
All         1296       915  2211

Accuracy: 0.966079

             precision    recall  f1-score   support

      legit       0.96      0.98      0.97      1271
   phishing       0.97      0.95      0.96       940

avg / total       0.97      0.97      0.97      2211

